# Causal Inference Homework 3
## Alex Pine (akp258@nyu.edu)
## February 26th

### 1a 
Perfect randomization, so ATE $= E[Y^1] - E[Y^0] = 1 - 0.5 = 0.5$

### 1b

The asymptotic value of $E[Y^1]$ and $E[Y^0]$ are not dependent on the exact size of the population. If 10% of students from both the control and treatment groups leave the program, we can treat them as if there were never in the program to begin with.  Since our estimation of ATE is based on the expected values of $Y^0$ and $Y^1$ of the population, 10% random attrition from both groups has no effect.  

### 1c

If students leave the program when their potential scores are greater than 1.6, then that puts an upper bound on $Y^0$ and $Y^1$. Since $Y^0$ was already bounded from above by 1, only $Y^1$ is affected. Now, $Y^1 \sim ([0,1.6])$.

ATE $= E[Y^1] - E[Y^0] = 0.8 - 0.5 = 0.3 $


### 1d

If the top 20% leave the program, then $Y^1 \sim U([0, 1.6])$ and $Y^0 \sim U([0, 0.8])$.

ATE $= E[Y1] - E[Y0] = \frac{1.6}{2} - \frac{0.8}{2} = 0.8 - 0.4 = 0.4$.



# Problem 2

Reducing the data down to only those who were in the same class size for all four years:

In [42]:
data = read.csv("/Users/pinesol/causal/hw3/krueger.csv")
# Take out the NAs. Leaves only 3085 rows out of 11598
data = data[complete.cases(subset(data, select=c('cltypek', 'cltype1', 'cltype2', 'cltype3'))),]
# Make sure the four class columns all have the same value in each row
data = data[data$cltypek == data$cltype1 | data$cltypek == data$cltype2 | data$cltypek == data$cltype3,]

# Remove NAs from scores
score_cols = c('tmathssk', 'tmathss1', 'tmathss2', 'tmathss3', 'treadssk', 'treadss1', 'treadss2', 'treadss3')
data[score_cols][is.na(data[score_cols])] <- 0

Computing the percentile scores for the students who were in in 'regular' or 'regular + aide' sized classes:

In [43]:
# ... for each grade, compute the percentile(ranging from 0 to 100) using only students 
# in 'regular size' and 'regular with aide size' class types.
percentile <- function(scores) {
    return(100 * rank(scores) / length(scores))
}

reg_rows = data[data['cltypek']  == 'regular class' | data['cltypek']  == 'regular + aide class',]

reg_math_k = percentile(reg_rows[['tmathssk']])
reg_read_k = percentile(reg_rows[['treadssk']])
reg_avg_k = (reg_math_k + reg_read_k) / 2
reg_math_1 = percentile(reg_rows[['tmathss1']])
reg_read_1 = percentile(reg_rows[['treadss1']])
reg_avg_1 = (reg_math_1 + reg_read_1) / 2
reg_math_2 = percentile(reg_rows[['tmathss2']])
reg_read_2 = percentile(reg_rows[['treadss2']])
reg_avg_2 = (reg_math_2 + reg_read_2) / 2
reg_math_3 = percentile(reg_rows[['tmathss3']])
reg_read_3 = percentile(reg_rows[['treadss3']])
reg_avg_3 = (reg_math_3 + reg_read_3) / 2

Computing the percentile score for students in small classes using the percentile distribution for the students in large classes:

In [44]:
# "Compute percentile score for students in small classes using the percentiles just computed".

small_percentiles <- function(small_scores, reg_scores) {
    small_percs = vector("list", length(small_scores))
    for (i in 1:length(small_scores)) {
        perc = 100*length(reg_scores[reg_scores <= small_scores[[i]]]) / length(reg_scores)
        small_percs[[i]] = perc
    }
    return(unlist(small_percs))
}

small_rows = data[data['cltypek']  == 'small class',]

small_math_k = small_percentiles(small_rows[['tmathssk']], reg_rows[['tmathssk']])
small_read_k = small_percentiles(small_rows[['treadssk']], reg_rows[['treadssk']])
small_avg_k = (small_math_k + small_read_k) / 2
small_math_1 = small_percentiles(small_rows[['tmathss1']], reg_rows[['tmathss1']])
small_read_1 = small_percentiles(small_rows[['treadss1']], reg_rows[['treadss1']])
small_avg_1 = (small_math_1 + small_read_1) / 2
small_math_2 = small_percentiles(small_rows[['tmathss2']], reg_rows[['tmathss2']])
small_read_2 = small_percentiles(small_rows[['treadss2']], reg_rows[['treadss2']])
small_avg_2 = (small_math_2 + small_read_2) / 2
small_math_3 = small_percentiles(small_rows[['tmathss3']], reg_rows[['tmathss3']])
small_read_3 = small_percentiles(small_rows[['treadss3']], reg_rows[['treadss3']])
small_avg_3 = (small_math_3 + small_read_3) / 2

Wrote a function to merge the percentile scores regular and small class sizes together so their order matches the order from the original data frame. I'm sure R has a way to do this, but I could not figure it out. 

In [45]:
merge_by_index <- function(a, b, a_index, b_index) {
    merged = c()
    i = 1
    j = 1
    while(i <= length(a_index) && j <= length(b_index)) {
        if (a_index[i] < b_index[j]) {
            merged = c(merged, a[i])
            i = i + 1
        } else {
            merged = c(merged, b[j])
            j = j + 1
        }
    }
    if (i <= length(a_index)) {
        merged = c(merged, a[i:length(a)])
    }
    if (j <= length(b_index)) {
        merged = c(merged, b[j:length(b)])
    } 
    return(merged)
}

merge_scores <- function(reg_scores, small_scores) {
    reg_index = as.numeric(row.names(reg_rows))
    small_index = as.numeric(row.names(small_rows))
    return(merge_by_index(reg_scores, small_scores, reg_index, small_index))
}

In [46]:
merged_k_scores = merge_scores(reg_avg_k, small_avg_k)
merged_1_scores = merge_scores(reg_avg_1, small_avg_1)
merged_2_scores = merge_scores(reg_avg_2, small_avg_2)
merged_3_scores = merge_scores(reg_avg_3, small_avg_3)

Performing the regression for kindergarden:

In [47]:
require(plm)

aide.f = factor(data['cltypek'] == 'regular + aide class')
small.f = factor(data['cltypek'] == 'small class')

summary(plm(merged_k_scores ~ aide.f + small.f, data=data, index=c('schidkn'), model='within'))

These series are constants and have been removed: stark, star1, star2, star3


Oneway (individual) effect Within Model

Call:
plm(formula = merged_k_scores ~ aide.f + small.f, data = data, 
    model = "within", index = c("schidkn"))

Unbalanced Panel: n=75, T=4-59, N=2043

Residuals :
   Min. 1st Qu.  Median 3rd Qu.    Max. 
 -60.20  -20.70    1.88   22.00   56.50 

Coefficients :
            Estimate Std. Error t-value Pr(>|t|)    
aide.fTRUE    0.3074     1.6559  0.1856   0.8527    
small.fTRUE   7.5747     1.4870  5.0941 3.84e-07 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Total Sum of Squares:    1471100
Residual Sum of Squares: 1444200
R-Squared:      0.01826
Adj. R-Squared: -0.019691
F-statistic: 18.2836 on 2 and 1966 DF, p-value: 1.3567e-08

1st grade regression:

In [48]:
aide.f = factor(data['cltype1'] == 'regular + aide class')
small.f = factor(data['cltype1'] == 'small class')

summary(plm(merged_1_scores ~ aide.f + small.f, data=data, index=c('schid1n'), model='within'))

These series are constants and have been removed: stark, star1, star2, star3


Oneway (individual) effect Within Model

Call:
plm(formula = merged_1_scores ~ aide.f + small.f, data = data, 
    model = "within", index = c("schid1n"))

Unbalanced Panel: n=75, T=4-58, N=2043

Residuals :
   Min. 1st Qu.  Median 3rd Qu.    Max. 
-61.200 -20.100   0.387  22.100  55.500 

Coefficients :
            Estimate Std. Error t-value  Pr(>|t|)    
aide.fTRUE   0.23075    1.61386  0.1430    0.8863    
small.fTRUE  9.69406    1.42821  6.7876 1.505e-11 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Total Sum of Squares:    1416500
Residual Sum of Squares: 1372000
R-Squared:      0.031429
Adj. R-Squared: -0.0060132
F-statistic: 31.8973 on 2 and 1966 DF, p-value: 2.3292e-14

2nd grade regression:

In [49]:
aide.f = factor(data['cltype2'] == 'regular + aide class')
small.f = factor(data['cltype2'] == 'small class')

summary(plm(merged_2_scores ~ aide.f + small.f, data=data, index=c('schid2n'), model='within'))

These series are constants and have been removed: stark, star1, star2, star3


Oneway (individual) effect Within Model

Call:
plm(formula = merged_2_scores ~ aide.f + small.f, data = data, 
    model = "within", index = c("schid2n"))

Unbalanced Panel: n=75, T=4-59, N=2043

Residuals :
   Min. 1st Qu.  Median 3rd Qu.    Max. 
 -59.90  -20.50    1.01   22.90   56.30 

Coefficients :
            Estimate Std. Error t-value  Pr(>|t|)    
aide.fTRUE    1.7089     1.6738  1.0210    0.3074    
small.fTRUE   7.2492     1.5126  4.7924 1.772e-06 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Total Sum of Squares:    1464900
Residual Sum of Squares: 1444400
R-Squared:      0.013945
Adj. R-Squared: -0.024173
F-statistic: 13.9019 on 2 and 1966 DF, p-value: 1.0111e-06

3rd grade regression:

In [51]:
aide.f = factor(data['cltype3'] == 'regular + aide class')
small.f = factor(data['cltype3'] == 'small class')

summary(plm(merged_3_scores ~ aide.f + small.f, data=data, index=c('schid3n'), model='within'))

These series are constants and have been removed: stark, star1, star2, star3


Oneway (individual) effect Within Model

Call:
plm(formula = merged_3_scores ~ aide.f + small.f, data = data, 
    model = "within", index = c("schid3n"))

Unbalanced Panel: n=75, T=6-56, N=2043

Residuals :
   Min. 1st Qu.  Median 3rd Qu.    Max. 
-60.100 -21.700   0.863  22.200  57.700 

Coefficients :
            Estimate Std. Error t-value  Pr(>|t|)    
aide.fTRUE    2.0787     1.7185  1.2096    0.2266    
small.fTRUE   8.1175     1.5776  5.1454 2.935e-07 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Total Sum of Squares:    1472700
Residual Sum of Squares: 1448200
R-Squared:      0.01667
Adj. R-Squared: -0.021343
F-statistic: 16.6646 on 2 and 1966 DF, p-value: 6.6577e-08

Looking at these statistics, it appears that small class size is a statistically significant predictor of test scores.